In [6]:
import sys

import pandas as pd
import rtree as rt

gtfstk_dir = '../../gtfstk/'
sys.path.append(gtfstk_dir)
import gtfstk as gt

data_dir = '../data/'

In [15]:
gt.timestr_to_seconds('11:30:00')
feed = gt.Feed('../data/cairns_gtfs.zip', dist_units_out='km')

In [17]:
def build_shapes_by_segment(self, geometry_by_segment, shape_buffer):
    """
    segment ID -> list of IDs of shapes such that 
                  the shape buffered by ``shape_buffer`` map units
                  contains the segment
    """
    assert self.segments is not None,\
      "Error: need the files segments.geojson and for this"

    # Get alignment shapes in UTM, excluding shapes 
    # not in self.frequencies, and buffer them
    d = self.get_linestring_by_shape(use_utm=True)     
    shapes = self.frequencies['shape_id'].unique()
    polygon_by_shape =\
      {shape: linestring.buffer(alignment_buffer) 
      for shape, linestring in d.items() if shape in shapes}

    # Create dict shape -> alignment IDs
    alignments_by_shape = {}
    for shape, group in self.frequencies[['shape_id', 'alignment_id']].\
      groupby('shape_id'):
        alignments_by_shape[shape] = list(group['alignment_id'].unique())

    # Build a spatial index with RTree that is 
    # based on buffered alignment shapes.
    # RTree indices must be integers, i believe.  
    # So we can't use alignment IDs, which might be strings, as indices. 
    # So use an auxiliary dictionary.
    sp_by_index = dict(zip(range(len(polygon_by_shape)), 
      polygon_by_shape.items()))
    idx = rt.index.Index()
    for i, (shape, polygon) in sp_by_index.items():
        idx.insert(i, polygon.bounds)

    # For each segment, record what alignment shapes contain it
    result = {}
    linestring_by_segment = self.get_linestring_by_segment(use_utm=True)
    for segment, linestring in linestring_by_segment.items():
        # Get bounding box
        segment_box = linestring.bounds
        shapes = [sp_by_index[i][0] 
          for i in idx.intersection(segment_box)
          if sp_by_index[i][1].contains(linestring)] 
        result[segment] = [alignment for shape in shapes 
          for alignment in alignments_by_shape[shape]]
    return result


['location_type',
 'wheelchair_boarding',
 'route_type',
 'direction_id',
 'stop_sequence',
 'wheelchair_accessible',
 'bikes_allowed',
 'pickup_type',
 'dropoff_type',
 'timepoint',
 'monday',
 'tuesday',
 'wednesday',
 'thursday',
 'friday',
 'saturday',
 'sunday',
 'exception_type',
 'payment_method',
 'transfers',
 'shape_pt_sequence',
 'exact_times',
 'transfer_type',
 'min_transfer_time']